In [6]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
#bash
topic_read = spark\
.read\
.format("kafka")\
.option("kafka.bootstrap.servers", "kafka:9092")\
.option("subscribe", "topic-kvspark")\
.load()



In [3]:
topic_read.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [14]:
topic_string = topic_read.select(col("key").cast("string"), col("value").cast("string"))
topic_string.show()

+----+--------------------+
| key|               value|
+----+--------------------+
|null|kafka-console-con...|
|null|             1, Msg1|
|null|             2, Msg2|
|null|             3, Msg3|
|null|             4, Msg4|
|null|             5, Msg5|
|null|             6, Msg6|
|null|             7, Msg7|
|null|             8, Msg8|
|null|              9,Msg9|
|   1|                Msg1|
|   2|                Msg2|
|   3|                Msg3|
|   4|                Msg4|
+----+--------------------+



In [15]:
#streaming
topic_read_stream = spark\
.readStream\
.format("kafka")\
.option("kafka.bootstrap.servers", "kafka:9092")\
.option("subscribe", "topic-kvspark")\
.load()

In [16]:
topic_read_stream.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [23]:
topic_string_stream = topic_read_stream.select(col("key").cast("string"), col("value").cast("string"))

In [26]:
topic_string_stream_output = topic_string_stream.writeStream\
.format("kafka")\
.option("kafka.bootstrap.servers", "kafka:9092")\
.option("topic", "topic-kvspark-output")\
.option("checkpointLocation","/user/paulo/kafka_checkpoint")\
.trigger(continuous="5 second")\
.start()